In [ ]:
!jupyter nbextension enable --py --sys-prefix ipyleaflet   #jupyter notebook
!jupyter labextension install jupyter-leaflet    #jupyter lab

In [136]:
import os
import sys
import time
from datetime import datetime
from datetime import timedelta
from itertools import chain
from itertools import cycle
import collections
from pprint import pprint
from io import StringIO
import pathlib

import numpy as np
import pandas as pd
from pandas.tools.plotting import parallel_coordinates
import matplotlib.pyplot as plt

#import urllib
#import webbrowser
#import codecs
from bs4 import BeautifulSoup
from xml.dom import minidom
from xml.etree import ElementTree as ET
#from lxml import etree as ET #Supports xpath syntax
import xmltodict 

import nltk
import re
import Levenshtein
import networkx as nx
import json

#from nominatim import Nominatim
#from mpl_toolkits.basemap import Basemap   #http://matplotlib.org/basemap/api/basemap_api.html
from matplotlib import cm
import geopy.geocoders as gg
from geopy.geocoders import Nominatim
import geopandas as gpd
import ipyleaflet
from ipyleaflet import Map, basemaps, basemap_to_tiles, DrawControl

#import plotly as py
#import plotly.graph_objs as go
#import plotly.figure_factory as ff

from IPython.display import Image, HTML, IFrame, FileLink, FileLinks #needed to render in notebook
from IPython.core.display import display

#import pydotplus #Install http://www.graphviz.org/ & Instal https://pypi.python.org/pypi/pydotplus

#https://stackoverflow.com/questions/2148119/how-to-convert-an-xml-string-to-a-dictionary-in-python
#https://github.com/martinblech/xmltodict
#https://www.datacamp.com/community/tutorials/python-xml-elementtree
#https://stackabuse.com/reading-and-writing-xml-files-in-python/
#https://codereview.stackexchange.com/questions/117810/parsing-of-xml-in-python-using-minidom-and-inserting-data-into-a-database
#https://stackoverflow.com/questions/749796/pretty-printing-xml-in-python

In [3]:
pwd

'C:\\Users\\rsouza\\Documents\\Python Scripts\\exploreAT-Topothek.NLP'

Reading the XML Metadata files:

In [4]:
readme =  './TEI-XML-2018/README_English.txt'
print((open(readme, encoding="latin-1").read()))


Content of the XML files
XML are partially enriched with location data
* The location data are available as a tree in the respective entries
* Location data are available for those entries specified in helper_tables\placeName-bis Freq 10-nur Ö+STir - Tabellenblatt1.csv
* The basis are complete matches in the tag placeName

The complete place hierarchy can be found in helper_tables\listPlace.xml. This resulted in helper_tables\listPlaceBySigle.xml

There could be manual corrections to the data.


Processing Information on Places

In [5]:
placesfile =  './TEI-XML-2018/helper_tables/listPlace.xml'
#print((open(placesfile, encoding="utf-8").read()))
placestree = ET.parse(placesfile)
placesroot = placestree.getroot()
set([elem.tag for elem in placesroot.iter()])

{'{http://www.tei-c.org/ns/1.0}Bundesland',
 '{http://www.tei-c.org/ns/1.0}Gemeinde',
 '{http://www.tei-c.org/ns/1.0}Großregion',
 '{http://www.tei-c.org/ns/1.0}Kleinregion',
 '{http://www.tei-c.org/ns/1.0}Kommentar',
 '{http://www.tei-c.org/ns/1.0}Ort',
 '{http://www.tei-c.org/ns/1.0}Sigle_DB',
 '{http://www.tei-c.org/ns/1.0}idno',
 '{http://www.tei-c.org/ns/1.0}listPlace',
 '{http://www.tei-c.org/ns/1.0}note',
 '{http://www.tei-c.org/ns/1.0}place',
 '{http://www.tei-c.org/ns/1.0}placeName',
 '{http://www.tei-c.org/ns/1.0}record'}

In [6]:
placesdict = xmltodict.parse((open(placesfile, encoding="utf-8").read()))
print(placesdict.keys())
print(placesdict['listPlace'].keys())

odict_keys(['listPlace'])
odict_keys(['@xmlns', 'place'])


In [7]:
placeslist = placesdict['listPlace']['place']
state_bundesland = [p['placeName'] for p in placeslist]
print(state_bundesland)

['W.', 'STir.', 'OTir.', 'NTir.', 'Kä.', 'St.', 'Sa.', 'OÖ', 'NÖ', 'Bgl.']


In [8]:
placeslist[0]['listPlace']['place']['listPlace']['place']['listPlace']['place']['listPlace']['place'][0]['record']
#placeslist[1]['listPlace']['place'][0]['listPlace']['place'][0]['listPlace']['place'][0]['listPlace']['place'][0]['record']
#placeslist[2]['listPlace']['place']['listPlace']['place'][0]['listPlace']['place'][0]['listPlace']['place']['record']
#placeslist[3]['listPlace']['place'][0]['listPlace']['place'][0]['listPlace']['place']['listPlace']['place']['record']

OrderedDict([('Ort', 'Altmannsdorf'),
             ('Gemeinde', 'Wien'),
             ('Kleinregion', 'W.'),
             ('Großregion', 'W.'),
             ('Bundesland', 'W.'),
             ('Sigle_DB', None),
             ('Kommentar', None)])

In [9]:
identified_places = collections.defaultdict(list)
def show_record(something):
    if isinstance(something, list):
        for elem in something:
            show_record(elem)
    elif isinstance(something, collections.OrderedDict) and ('record' in something.keys()):
        try:
            #print(something['record']['Bundesland'], ' --> ', something['record']['Ort'])
            identified_places[something['record']['Bundesland']].append(something['record']['Ort'])
        except:
            #print(something['record'][0]['Bundesland'], ' --> ', something['record'][0]['Ort'])
            identified_places[something['record'][0]['Bundesland']].append(something['record'][0]['Ort'])
    elif isinstance(something, collections.OrderedDict):
        new = something['listPlace']['place']
        show_record(new)
    else:
        print(type(something))
        print(something)

In [10]:
show_record(placeslist)

In [11]:
identified_places.keys()

dict_keys(['W.', 'STir.', 'OTir.', 'NTir.', 'Kä.', 'St.', 'Sa.', 'OÖ', 'NÖ', 'Bgl.'])

In [12]:
df_places = pd.DataFrame(columns=['Bundesland', 'Ort'])
for key in identified_places.keys():
    df_temp = pd.DataFrame(identified_places[key], columns=['Ort'])
    df_temp['Bundesland'] = key
    df_places = pd.concat([df_places, df_temp], axis=0, sort=True)
df_places.head(30)

,Bundesland,Ort
0,W.,Altmannsdorf
1,W.,Aspern
2,W.,Atzgersdorf
3,W.,Breitensee
4,W.,Favoriten
5,W.,Gersthof
6,W.,Hernals
7,W.,Hietzing
8,W.,Inzersdorf
9,W.,Kalksburg


Using a geolocator to get the Coordinates

In [13]:
geolocator = gg.Nominatim(user_agent="test_application")

In [14]:
logradouro = identified_places['Bgl.'][0]
location = geolocator.geocode(logradouro)
print(location.address)
address = location.address
print((location.latitude, location.longitude))
latitude, longitude = location.latitude, location.longitude
print(location.raw)

Deutsch Jahrndorf, Bezirk Neusiedl, Burgenland, 2423, Österreich
(48.0116888, 17.1069611)
{'place_id': '197784614', 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'relation', 'osm_id': '303140', 'boundingbox': ['47.9708775', '48.031224', '17.053944', '17.160776'], 'lat': '48.0116888', 'lon': '17.1069611', 'display_name': 'Deutsch Jahrndorf, Bezirk Neusiedl, Burgenland, 2423, Österreich', 'class': 'boundary', 'type': 'administrative', 'importance': 0.523562468688337, 'icon': 'https://nominatim.openstreetmap.org/images/mapicons/poi_boundary_administrative.p.20.png'}


The next cells were used just for getting geographic information. The saved Pickle shall be used for the subsequent processing. 

In [124]:
def get_coordinates(place):
    try:
        location = geolocator.geocode(place)
        if not location:
            try:
                location = geolocator.geocode(place.split(',')[0])
            except:
                return None
    except:
        return None
    print(location)
    time.sleep(5)
    if location:
        return (location.longitude, location.latitude), location.address
    else:
        return None

In [49]:
df_places = pd.read_pickle("./df_places.pkl")
df_places.head(10)

,Bundesland,Ort,Coordinates,Adresses
0,W.,Altmannsdorf,"(10.4342455, 49.9358878)","Altmannsdorf, Gerolzhofen (VGem), Landkreis Sc..."
1,W.,Aspern,"(16.4847508, 48.2181854)","Aspern, Donaustadt, Wien, 1220, Österreich"
2,W.,Atzgersdorf,"(16.2954875, 48.1469838)","Atzgersdorf, Liesing, Wien, 1230, Österreich"
3,W.,Breitensee,"(10.5435061, 50.3531716)","Breitensee, Bad Königshofen i. Grabfeld (VGem)..."
4,W.,Favoriten,"(16.3828218364771, 48.1529937)","Favoriten, Wien, 1100, Österreich"
5,W.,Gersthof,None,None
6,W.,Hernals,"(16.2842142186391, 48.2354031)","Hernals, Wien, 1170, Österreich"
7,W.,Hietzing,"(16.2529858155656, 48.17854095)","Hietzing, Wien, 1130, Österreich"
8,W.,Inzersdorf,"(16.3527158778638, 48.14397215)","KG Inzersdorf, Liesing, Wien, 1230, Österreich"
9,W.,Kalksburg,"(16.2457624, 48.1376728)","Kalksburg, Liesing, Wien, 1230, Österreich"


In [100]:
df_places.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2981 entries, 0 to 2980
Data columns (total 4 columns):
Bundesland     2981 non-null object
Ort            2981 non-null object
Coordinates    2951 non-null object
Adresses       2952 non-null object
dtypes: object(4)
memory usage: 93.2+ KB


In [101]:
df_places.reset_index(inplace=True, drop=True)

In [122]:
df_places[df_places.Coordinates.isnull()]

,Bundesland,Ort,Coordinates,Adresses
19,STir.,Reschen am See,None,None
215,OTir.,Lavant,None,None
290,NTir.,Feichten im Kaunertal (Feuchten),None,None
344,NTir.,Freundsheim,None,None
345,NTir.,Untermieming $Untermiening$,None,None
353,NTir.,$Silltall; Anm.: inoffizielle Bezeichnung für ...,None,None
446,NTir.,Thiersee $Tiersee$,None,None
484,NTir.,Aschau bei Kirchberg,None,None
552,Kä.,Nölbing,None,None
571,Kä.,Reisach im Gailtail,None,None


In [121]:
location = geolocator.geocode('Scheiblingskirchen')
print(location.address)
address = location.address
print((location.latitude, location.longitude))
latitude, longitude = location.latitude, location.longitude
print(location.raw)

AttributeError: 'NoneType' object has no attribute 'address'

In [123]:
df_places[df_places.Coordinates.isnull()].index

Int64Index([  19,  215,  290,  344,  345,  353,  446,  484,  552,  571,  580,
             830,  854, 1037, 1199, 1241, 1281, 1293, 1332, 1615, 1773, 1865,
            1984, 2008, 2012, 2265, 2305, 2566, 2971],
           dtype='int64')

In [125]:
geoinfo_missing = []
for ort in df_places[df_places.Coordinates.isnull()].Ort:
    geoinfo_missing.append(get_coordinates(ort))

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [126]:
print(len(geoinfo_missing))
print(len(df_places[df_places.Coordinates.isnull()]))

29
29


In [106]:
idx = df_places[df_places.Coordinates.isnull()].index
df_places.loc[idx, 'Coordinates'] = [x[0] if x else None for x in geoinfo_missing]

In [107]:
df_places[df_places.Adresses.isnull()]

,Bundesland,Ort,Coordinates,Adresses
19,STir.,Reschen am See,None,None
215,OTir.,Lavant,None,None
290,NTir.,Feichten im Kaunertal (Feuchten),None,None
344,NTir.,Freundsheim,None,None
345,NTir.,Untermieming $Untermiening$,None,None
353,NTir.,$Silltall; Anm.: inoffizielle Bezeichnung für ...,None,None
446,NTir.,Thiersee $Tiersee$,None,None
484,NTir.,Aschau bei Kirchberg,None,None
552,Kä.,Nölbing,None,None
571,Kä.,Reisach im Gailtail,None,None


In [127]:
geoinfo_missing = []
for ort in df_places[df_places.Adresses.isnull()].Ort:
    geoinfo_missing.append(get_coordinates(ort))

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [128]:
print(len(geoinfo_missing))
print(len(df_places[df_places.Adresses.isnull()]))

29
29


In [110]:
idx = df_places[df_places.Adresses.isnull()].index
df_places.loc[idx, 'Adresses'] = [x[1] if x else None for x in geoinfo_missing]

In [129]:
df_places.loc[df_places[df_places.Coordinates.isnull()].index]

,Bundesland,Ort,Coordinates,Adresses
19,STir.,Reschen am See,None,None
215,OTir.,Lavant,None,None
290,NTir.,Feichten im Kaunertal (Feuchten),None,None
344,NTir.,Freundsheim,None,None
345,NTir.,Untermieming $Untermiening$,None,None
353,NTir.,$Silltall; Anm.: inoffizielle Bezeichnung für ...,None,None
446,NTir.,Thiersee $Tiersee$,None,None
484,NTir.,Aschau bei Kirchberg,None,None
552,Kä.,Nölbing,None,None
571,Kä.,Reisach im Gailtail,None,None


In [130]:
df_places.to_pickle("./df_places_plus.pkl")

In [135]:
#https://docs.python.org/2.4/lib/standard-encodings.html
df_places.to_csv("./df_places.csv", encoding='utf_16')

Showing places in a map

In [140]:
m = Map(
    layers=(basemap_to_tiles(basemaps.NASAGIBS.ModisTerraTrueColorCR, "2018-04-08"), ),
    center=(52.204793, 360.121558),
    zoom=4)

In [142]:
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [25]:
#https://developers.google.com/maps/documentation/staticmaps/
#http://stackoverflow.com/questions/2660201/what-parameters-should-i-use-in-a-google-maps-url-to-go-to-a-lat-lon/9919251#9919251
#m – normal map k – satellite h – hybrid p – terrain

def gmap(address,lat,lon,zoom=15,tmap='m'):
    # Google Maps URL template for an iframe
    google_maps_url = 'http://maps.google.com/maps?q={0}&loc:{1}+{2}&z={3}&t={4}&output=embed'.format(address, lat, lon, zoom, tmap,)
    display(IFrame(google_maps_url, '800px', '600px'))
    
gmap(address, latitude, longitude,10,'k')

Reading the XML Data files:

In [12]:
xmlfile =  './TEI-XML-2018/bel_d-tei-02/d157_qdb-TEI-02.xml'
#xmlfile = open(xmlfile, mode='r', encoding='utf-8').read()
namespaces = {'TEI': 'http://www.tei-c.org/ns/1.0'} # add more as needed

Parsing the XML tree

In [13]:
xmldoc = minidom.parse(xmlfile)  #xmldoc = ET.XML(xmlfile)
bs = BeautifulSoup(open(xmlfile, encoding="utf8"), 'xml')
tree = ET.parse(xmlfile)

In [14]:
root = tree.getroot()
print(root)

<Element '{http://www.tei-c.org/ns/1.0}TEI' at 0x7fe12102a458>


In [15]:
my_namespaces = dict([node for _, node in ET.iterparse(StringIO(open(xmlfile, mode='r', encoding='utf-8').read()), events=['start-ns'])])
print(my_namespaces)

{'': 'http://www.tei-c.org/ns/1.0'}


In [16]:
for child in root:
    print(child.tag, child.attrib)

{http://www.tei-c.org/ns/1.0}teiHeader {}
{http://www.tei-c.org/ns/1.0}text {}


In [17]:
set([elem.tag for elem in root.iter()])

{'{http://www.tei-c.org/ns/1.0}TEI',
 '{http://www.tei-c.org/ns/1.0}att',
 '{http://www.tei-c.org/ns/1.0}bibl',
 '{http://www.tei-c.org/ns/1.0}body',
 '{http://www.tei-c.org/ns/1.0}certainty',
 '{http://www.tei-c.org/ns/1.0}cit',
 '{http://www.tei-c.org/ns/1.0}date',
 '{http://www.tei-c.org/ns/1.0}def',
 '{http://www.tei-c.org/ns/1.0}edition',
 '{http://www.tei-c.org/ns/1.0}editionStmt',
 '{http://www.tei-c.org/ns/1.0}editorialDecl',
 '{http://www.tei-c.org/ns/1.0}encodingDesc',
 '{http://www.tei-c.org/ns/1.0}entry',
 '{http://www.tei-c.org/ns/1.0}etym',
 '{http://www.tei-c.org/ns/1.0}fileDesc',
 '{http://www.tei-c.org/ns/1.0}form',
 '{http://www.tei-c.org/ns/1.0}geoDecl',
 '{http://www.tei-c.org/ns/1.0}gi',
 '{http://www.tei-c.org/ns/1.0}gram',
 '{http://www.tei-c.org/ns/1.0}gramGrp',
 '{http://www.tei-c.org/ns/1.0}idno',
 '{http://www.tei-c.org/ns/1.0}interp',
 '{http://www.tei-c.org/ns/1.0}label',
 '{http://www.tei-c.org/ns/1.0}listPlace',
 '{http://www.tei-c.org/ns/1.0}listPrefixDe

In [18]:
places = []
for elem in root.iter('{http://www.tei-c.org/ns/1.0}place'):
    # print(elem.attrib)
    # print(elem.attrib['type'])
    places.append(elem.attrib['type'])
freq_places = Counter(places)
freq_places.most_common()

[('Bundesland', 891),
 ('Großregion', 797),
 ('Kleinregion', 783),
 ('Gemeinde', 696),
 ('Ort', 696)]

Dictionary Approach

In [19]:
xml2dictfile = xmltodict.parse((open(xmlfile, mode='r', encoding='utf-8').read()))

In [20]:
print(xml2dictfile['TEI'].keys())

odict_keys(['@xmlns', 'teiHeader', 'text'])


In [21]:
xml2dictfile['TEI']['@xmlns']
#xml2dictfile['TEI']['teiHeader']

'http://www.tei-c.org/ns/1.0'

In [22]:
print(len(xml2dictfile['TEI']['text']['body']['entry']))
xml2dictfile['TEI']['text']['body']['entry'][0].keys()

1991


odict_keys(['@xml:id', '@xml:lang', 'form', 'gramGrp', 'sense', 'cit', 'note', 'ref', 'etym', 'usg'])

In [23]:
xml2dictfile['TEI']['text']['body']['entry'][0]

OrderedDict([('@xml:id', 'd157_qdb-d1e2'),
             ('@xml:lang', 'bar'),
             ('form',
              [OrderedDict([('@type', 'hauptlemma'), ('orth', 'Tin')]),
               OrderedDict([('@type', 'lautung'),
                            ('@n', '1'),
                            ('pron',
                             [OrderedDict([('@notation', 'tustep'),
                                           ('#text', '|A t~-in')]),
                              OrderedDict([('@notation', 'ipa'),
                                           ('@resp', '#JB'),
                                           ('@change', '01'),
                                           ('#text', '|A t~-in')])]),
                            ('gramGrp', OrderedDict([('gram', '[m,sg+U]')]))]),
               OrderedDict([('@type', 'lautung'),
                            ('@n', '2'),
                            ('pron',
                             [OrderedDict([('@notation', 'tustep'),
                              

In [31]:
for idx, entry in enumerate(xml2dictfile['TEI']['text']['body']['entry']):
    #print(idx)
    places = []
    lemmas = []
    forms = []
    if 'usg' in entry.keys():
        if isinstance(entry['usg'], list):
            for local in entry['usg']:
                try:
                    #print(local['placeName']['#text'])
                    places.append(local['placeName']['#text'])
                except:
                    #print(local['placeName'])
                    places.append(local['placeName'])
        else:
            places = []
            try:
                #print(entry['usg']['placeName']['#text']);
                places.append(entry['usg']['placeName']['#text'])
            except:
                #print(entry['usg']['placeName']);
                places.append(entry['usg']['placeName'])
                
    if 'form' in entry.keys():            
        if isinstance(entry['form'], list):
            for elem in entry['form']:
                if elem['@type'] == 'hauptlemma' and isinstance(elem['orth'], str):          
                    #print(elem['orth'])
                    lemmas.append(elem['orth'])
                elif elem['@type'] == 'hauptlemma' and isinstance(elem['orth'], list):
                    for subelem in elem['orth']:
                        if subelem['@type'] == 'normalized':
                            #print(subelem['#text'])
                            lemmas.append(subelem['#text'])
        else:
            if isinstance(entry['form']['orth'], list):
                for elem in entry['form']['orth']:
                    if elem['@type'] == 'normalized':
                        #print(elem['#text'])
                        lemmas.append(elem['#text'])
            else:
                #print(entry['form']['orth'])
                lemmas.append(entry['form']['orth'])
    #if 'cit' in entry.keys():
    #    print(entry['cit']) #['ref'])
    #    print(entry['cit'].keys())
  
    print(lemmas, places)

['Tin'] ['Matrei/O. OTir.']
['Häutin'] ['Matrei/O. OTir.']
['Gesotttin'] ['Matrei/O. OTir.']
['Stadel'] ['Matrei/O. OTir.']
['Tin'] ['Matrei/O. OTir.']
['Stadeltor'] ['Matrei/O. OTir.']
['Tina'] ['Salurn']
['Tina'] ['Salurn']
['Dinamít'] ['(Umg.) Herzogenb-g. NÖ']
['Dinamít'] []
['Dinamít'] []
['Dinamít'] []
['Dinamo'] []
['Dinamo'] ['(Umg.) Herzogenb-g. NÖ']
['Dinamo'] ['Königswerth Egerld.']
['Tina'] ['Marling']
['Tina'] ['Marling']
['Tina'] ['Marling']
['Dinastí'] ['(Umg.) Herzogenb-g. NÖ']
['dinástisch'] ['(Umg.) Herzogenb-g. NÖ']
['Tinazia'] []
['tindern'] []
['tindern'] []
['tindern'] []
['tindern'] ['Malta Kä.']
['tindern'] ['Malta Kä.']
['schebern'] ['Malta Kä.']
['scheppern'] ['Malta Kä.']
['tuschen'] ['Malta Kä.']
['Tinef'] ['Wien']
['Tinef'] ['Wien']
['Tinef'] []
['Tinef'] []
['Tinef'] []
['Tinef'] []
['Tinef'] []
['Tinef'] []
['Tinef'] []
['Tinef'] []
['Tinef'] []
['Tinef'] ['Wien']
['Schmarren'] ['Wien']
['Drëck'] ['Wien']
['Tinef'] []
['Tinéll'] []
['Tinellein'] ['Böwald'

In [ ]:
places = []
lemmas = []
forms = []
for idx, entry in enumerate(xml2dictfile['TEI']['text']['body']['entry']):
    #print(idx)
    if 'usg' in entry.keys():
        if isinstance(entry['usg'], list):
            for local in entry['usg']:
                try:
                    #print(local['placeName']['#text'])
                    places.append(local['placeName']['#text'])
                except:
                    #print(local['placeName'])
                    places.append(local['placeName'])
        else:
            places = []
            try:
                #print(entry['usg']['placeName']['#text']);
                places.append(entry['usg']['placeName']['#text'])
            except:
                #print(entry['usg']['placeName']);
                places.append(entry['usg']['placeName'])
                
    if 'form' in entry.keys():            
        if isinstance(entry['form'], list):
            for elem in entry['form']:
                if elem['@type'] == 'hauptlemma' and isinstance(elem['orth'], str):          
                    #print(elem['orth'])
                    lemmas.append(elem['orth'])
                elif elem['@type'] == 'hauptlemma' and isinstance(elem['orth'], list):
                    for subelem in elem['orth']:
                        if subelem['@type'] == 'normalized':
                            #print(subelem['#text'])
                            lemmas.append(subelem['#text'])
        else:
            if isinstance(entry['form']['orth'], list):
                for elem in entry['form']['orth']:
                    if elem['@type'] == 'normalized':
                        #print(elem['#text'])
                        lemmas.append(elem['#text'])
            else:
                #print(entry['form']['orth'])
                lemmas.append(entry['form']['orth'])
    #if 'cit' in entry.keys():
    #    print(entry['cit']) #['ref'])
    #    print(entry['cit'].keys())
  
    print(lemmas, places)